In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
import tensorflow as tf

In [12]:
# Definisikan focal_loss_fixed
def focal_loss(gamma=2., alpha=4.):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true = tf.cast(y_true, tf.float32)

        alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (tf.ones_like(y_true) - y_true) * (tf.ones_like(y_true) - y_pred)
        fl = - alpha_t * tf.pow((tf.ones_like(y_true) - p_t), gamma) * tf.math.log(p_t)
        return tf.reduce_mean(fl)
    return focal_loss_fixed

In [13]:
# Tambahkan fungsi kustom ke dalam custom objects
get_custom_objects().update({'focal_loss_fixed': focal_loss()})

In [14]:
# Muat model dengan custom_objects
model1 = load_model('Model1_DermaOne.h5', custom_objects={'focal_loss_fixed': focal_loss()})
model2 = load_model('Model2_DermaOne.h5', custom_objects={'focal_loss_fixed': focal_loss()})

In [15]:
# Define input shape
input_shape = model1.input_shape[1:]

# Create input layer
input_layer = Input(shape=input_shape)

# Get outputs from both models
output1 = model1(input_layer)
output2 = model2(input_layer)

# Concatenate the outputs
combined_output = Concatenate()([output1, output2])

# Create and compile ensemble model
ensemble_model = Model(inputs=input_layer, outputs=combined_output)
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Save the ensemble model in HDF5 format
ensemble_model.save('Model_DermaOne.h5')

c:\Users\User\AppData\Local\python\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
